<a href="https://colab.research.google.com/github/rikdantas/Aprendizagem-de-Maquinas/blob/main/IMD1101/Pratica_02/Pratica02_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prática 02 - Técnica Supervisionada (DT)
Nesse notebook será aplicado o método de árvore de decisão nas 6 melhores bases que eu tinha encontrado na prática 01 e também nas 6 bases que foram aplicados o PCA também na prática 01.

Aluno: Paulo Ricardo Dantas

In [9]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

## Baixando os datasets

In [2]:
# Criando função para baixar os datasets, assim deixando o código um pouco mais organizado
def baixar_dataset(url, output):
    gdown.download(url, output, quiet=True)

In [3]:
# Baixando datasets do VGG16
baixar_dataset('https://drive.google.com/uc?id=1Lig0-UqpGvMQCmV5r5U__yucoocrlbtf', 'VGG16_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1V7_N11eqBnTiM9iKY3vhMVrTzoERrijS', 'VGG16_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1d8spY-o4xBsozdeSbGGlweZrips1m0HC', 'VGG16_3.csv')

# Baixando datasets do VGG19
baixar_dataset('https://drive.google.com/uc?id=1yjuqQKxXtBqlMetGgD3fyFjDZ3zg3YDS', 'VGG19_1.csv')
baixar_dataset('https://drive.google.com/uc?id=13nmb2kPrv6oVKlhodM-_ierzsMrvWL18', 'VGG19_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1qB2SQXLtgS01FqTrwGYLdOY1pwRqkuXu', 'VGG19_3.csv')

# Baixando datasets do PCA
baixar_dataset('https://drive.google.com/uc?id=14sGLGGak-1WZwnLrqEMEZbYWNZL1dIVn', 'PCA_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1DCXErsoiqwcM59QkIOdlZcIVGZDmWYTK', 'PCA_2.csv')
baixar_dataset('https://drive.google.com/uc?id=15k9bMm1ZfliNeHQ6MX7SlEy49UQJrxtv', 'PCA_3.csv')
baixar_dataset('https://drive.google.com/uc?id=17BJNdaH_8fCYTJn-l-QH7ayRaLuWYxQ6', 'PCA_4.csv')
baixar_dataset('https://drive.google.com/uc?id=18_qRzZ4wA-laBL0bsCch6xTus7SYLjLf', 'PCA_5.csv')
baixar_dataset('https://drive.google.com/uc?id=1mc9uT_EDkh9xC8rWnZTUZp6qX8Q7verz', 'PCA_6.csv')

## Importando os datasets
Vamos usar o pandas para importar os datasets para DataFrames. Usaremos os mesmos nomes que foram baixados os arquivos dos datasets. Podemos relacionar os datasets com as configurações como visto na tabela a seguir:

| Nome do dataset | Configuração            |
|-----------------|-------------------------|
| VGG16_1         | CNN_VGG16_128_avg       |
| VGG16_2         | CNN_VGG16_128_max       |
| VGG16_3         | CNN_VGG16_256_avg       |
| VGG19_1         | CNN_VGG19_128_avg       |
| VGG19_2         | CNN_VGG19_128_max       |
| VGG19_3         | CNN_VGG19_256_avg       |
| PCA_1           | PCA_CNN_VGG16_128_avg   |
| PCA_2           | PCA_CNN_VGG16_128_max   |
| PCA_3           | PCA_CNN_VGG16_256_avg   |
| PCA_4           | PCA_CNN_VGG19_128_avg   |
| PCA_5           | PCA_CNN_VGG19_128_max   |
| PCA_6           | PCA_CNN_VGG19_256_avg   |


In [4]:
# Importando os datasets do VGG16
vgg16_1 = pd.read_csv('VGG16_1.csv', encoding='utf-8')
vgg16_2 = pd.read_csv('VGG16_2.csv', encoding='utf-8')
vgg16_3 = pd.read_csv('VGG16_3.csv', encoding='utf-8')

# Importando os datasets do VGG19
vgg19_1 = pd.read_csv('VGG19_1.csv', encoding='utf-8')
vgg19_2 = pd.read_csv('VGG19_2.csv', encoding='utf-8')
vgg19_3 = pd.read_csv('VGG19_3.csv', encoding='utf-8')

# Importando os datasets do PCA
pca_1 = pd.read_csv('PCA_1.csv', encoding='utf-8')
pca_2 = pd.read_csv('PCA_2.csv', encoding='utf-8')
pca_3 = pd.read_csv('PCA_3.csv', encoding='utf-8')
pca_4 = pd.read_csv('PCA_4.csv', encoding='utf-8')
pca_5 = pd.read_csv('PCA_5.csv', encoding='utf-8')
pca_6 = pd.read_csv('PCA_6.csv', encoding='utf-8')

## Aplicando a DecisionTree


### Utilizando holdout


In [5]:
# Lista dos datasets
datasets = {
    "vgg16_1": vgg16_1,
    "vgg16_2": vgg16_2,
    "vgg16_3": vgg16_3,
    "vgg19_1": vgg19_1,
    "vgg19_2": vgg19_2,
    "vgg19_3": vgg19_3,
    "pca_1": pca_1,
    "pca_2": pca_2,
    "pca_3": pca_3,
    "pca_4": pca_4,
    "pca_5": pca_5,
    "pca_6": pca_6
}

results = []

In [6]:
# Loop pelos datasets
for name, data in datasets.items():
    # Separando atributos e rótulos
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

    # Dividindo em treino e teste (70/30)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    # Testando diferentes valores de max_depth
    for max_depth in range(2, 11):
        # Criando e treinando o modelo
        model = DecisionTreeClassifier(max_depth=max_depth, random_state=1)
        model.fit(X_train, y_train)

        # Prevendo no conjunto de teste
        y_pred = model.predict(X_test)

        # Calculando a acurácia
        acc = accuracy_score(y_test, y_pred)

        # Salvando os resultados
        results.append({"Dataset": name, "Max Depth": max_depth, "Accuracy": acc})

# Convertendo os resultados para DataFrame
results_df = pd.DataFrame(results)

# Salvando os resultados em um CSV
results_df.to_csv("Resultados_Decision_Tree_Holdout.csv", index=False)

# Exibindo os resultados
print(results_df)

     Dataset  Max Depth  Accuracy
0    vgg16_1          2  0.654167
1    vgg16_1          3  0.700000
2    vgg16_1          4  0.691667
3    vgg16_1          5  0.687500
4    vgg16_1          6  0.679167
..       ...        ...       ...
103    pca_6          6  0.637500
104    pca_6          7  0.645833
105    pca_6          8  0.633333
106    pca_6          9  0.633333
107    pca_6         10  0.650000

[108 rows x 3 columns]


### Utilizando K-fold


In [7]:
# Lista dos datasets
datasets = {
    "vgg16_1": vgg16_1,
    "vgg16_2": vgg16_2,
    "vgg16_3": vgg16_3,
    "vgg19_1": vgg19_1,
    "vgg19_2": vgg19_2,
    "vgg19_3": vgg19_3,
    "pca_1": pca_1,
    "pca_2": pca_2,
    "pca_3": pca_3,
    "pca_4": pca_4,
    "pca_5": pca_5,
    "pca_6": pca_6
}

# Resultados do k-fold
kfold_results = []

In [8]:
# Loop pelos datasets
for name, data in datasets.items():
    # Separando atributos e rótulos
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values

    # Configurando o KFold
    kf = KFold(n_splits=10, shuffle=True, random_state=1)

    # Testando diferentes valores de max_depth
    for max_depth in range(2, 11):
        fold_accuracies = []

        # Validação cruzada
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Criando e treinando o modelo
            model = DecisionTreeClassifier(max_depth=max_depth, random_state=1)
            model.fit(X_train, y_train)

            # Prevendo no conjunto de teste
            y_pred = model.predict(X_test)

            # Calculando a acurácia do fold
            acc = accuracy_score(y_test, y_pred)
            fold_accuracies.append(acc)

        # Calculando a média das acurácias dos folds
        mean_acc = np.mean(fold_accuracies)

        # Salvando os resultados
        kfold_results.append({"Dataset": name, "Max Depth": max_depth, "Mean Accuracy": mean_acc})

# Convertendo os resultados para DataFrame
kfold_results_df = pd.DataFrame(kfold_results)

# Salvando resultado em CSV
kfold_results_df.to_csv("Resultados_Decision_Tree_Kfold.csv", index=False)

# Visualizando os resultados
print(kfold_results_df)

     Dataset  Max Depth  Mean Accuracy
0    vgg16_1          2        0.67375
1    vgg16_1          3        0.67250
2    vgg16_1          4        0.68625
3    vgg16_1          5        0.65625
4    vgg16_1          6        0.67250
..       ...        ...            ...
103    pca_6          6        0.66250
104    pca_6          7        0.64125
105    pca_6          8        0.64125
106    pca_6          9        0.62750
107    pca_6         10        0.62750

[108 rows x 3 columns]
